In [1]:
import xarray as xr
import numpy as np
import glob, os
from shutil import copyfile

# User-supplied

In [8]:
elevinfile = '/glade/p/work/manab/ff/islandpark/input/us_09066300_elev_bands.nc'
elevoutfile = '/glade/p/work/manab/ff/islandpark/input/ip_elev_bands.nc'
forcdir = '/glade/p/work/manab/ff/islandpark/inputnew'

# Change Latitude, Longitude

In [16]:
elev = xr.open_dataset(elevinfile)
elev['latitude'] = elev['latitude']*0 + 44.5118087293063
elev['longitude'] = elev['longitude']*0 -111.434567218585
elev = elev.sel(elevation_band = slice(None, 4))
print(elev)

<xarray.Dataset>
Dimensions:         (elevation_band: 4, latitude: 1, longitude: 1)
Coordinates:
  * longitude       (longitude) float64 -111.4
  * latitude        (latitude) float64 44.51
  * elevation_band  (elevation_band) int32 1 2 3 4
Data variables:
    area_frac       (elevation_band, latitude, longitude) float64 ...
    mean_elev       (elevation_band, latitude, longitude) float64 ...
    prec_frac       (elevation_band, latitude, longitude) float64 ...


# Area/Precip Fractions and mean_elev

In [17]:
lat = elev['latitude'].values
long = elev['longitude'].values
elevation_band = elev['elevation_band'].values

mean_elev = np.array([2250, 2550, 2550, 3350]).reshape((4, 1, 1))

#The 1-X is to ensure that sum of fractions is equal to 1.
frac = np.array([0.65418, 0.099, 0.233, (1-(0.65418 + 0.099 + 0.233))]).reshape((4, 1, 1))  
me = xr.DataArray(mean_elev, coords={'latitude': lat, 'longitude': long, 
                                'elevation_band': elevation_band},
             dims=['elevation_band', 'latitude', 'longitude'])

fr = xr.DataArray(frac, coords={'latitude': lat, 'longitude': long, 
                                'elevation_band': elevation_band},
             dims=['elevation_band', 'latitude', 'longitude'])

elev['mean_elev'] = me
elev['area_frac'] = fr
elev['prec_frac'] = fr
elev.to_netcdf(elevoutfile)

# Create copies of elevation bands for each ensemble

In [18]:
forcfiles = glob.glob(forcdir + '/*nc')
forcfiles = [x for x in forcfiles if "elev_bands" not in x]
forcfiles = sorted(forcfiles)

for count, value in enumerate(forcfiles):
    outfile = os.path.join(forcdir, os.path.basename(value).split('.')[0] + '_elev_bands.nc')
    copyfile(elevoutfile, outfile)
